In [40]:
# Import functions #
import argparse,datetime,os,sys,time

try:
    import plotly.graph_objects as go
except:
    go = None  

if go is None:
    print("plotly is not installed")

import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets

from tools import load_groupfile, query_api, filterIQM, merge_dfs, make_vio_plot

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [41]:
#define widgets
modality_widget=widgets.Dropdown(
    options=['bold', 'T1w', 'T2w'],
    description='Modality:',
    disabled=False)

TR=widgets.FloatRangeSlider(
    min=0,
    max=5,
    step=0.1,
    description='TR Range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)

TE=widgets.FloatRangeSlider(
    min=0,
    max=2,
    step=0.1,
    description='TE Range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)
FD = widgets.FloatText(description="max FD",disabled=False)

SNR= widgets.FloatText(description="min SNR",  disabled=False)

IQM_bold=widgets.SelectMultiple(
    options=['fwhm_avg','fd_mean', 'aor', 'aqi', 'efc'],
    description='IQM to plot',
    disabled=False)

IQM_structural=widgets.SelectMultiple(
    options=['fwhm_avg','cnr', 'efc', 'snr'],
    description='IQM to plot',
    disabled=False)


select_bold=widgets.SelectMultiple(
    options=['TR', 'FD', 'SNR'],
    description='Parameters',
    disabled=False)

#need to make separate filters for T1 and T2
select_structural=widgets.SelectMultiple(
    options=['TR'],
    description='Parameters',
    disabled=False)

In [42]:
display(modality_widget)

Dropdown(description='Modality:', options=('bold', 'T1w', 'T2w'), value='bold')

In [44]:
modal=modality_widget.value
if modal == "bold":
    display(select_bold)
    display(IQM_bold)
else:
    display(select_structural)
    display(IQM_structural)

SelectMultiple(description='Parameters', options=('TR', 'FD', 'SNR'), value=())

SelectMultiple(description='IQM to plot', options=('fwhm_avg', 'fd_mean', 'aor', 'aqi', 'efc'), value=())

In [58]:
if modal == "bold":
    IQM_to_plot=IQM_bold.value
    for i in select_bold.value:
        display(eval(i))
else:
    for i in select_structural.value:
        IQM_to_plot=IQM_structural.value
        display(eval(i))

FloatRangeSlider(value=(2.1, 4.3), continuous_update=False, description='TR Range:', max=5.0, readout_format='…

FloatText(value=0.5, description='max FD')

FloatText(value=3.0, description='min SNR')

In [59]:
if modal == "bold":
    IQM_to_plot=IQM_bold.value
    to_filter=select_bold.value
else:
    IQM_to_plot=IQM_structural.value
    to_filter=select_structural.value

In [60]:
filter_list=[]
filter_dict={'TR': "TR > {}".format(TR.value[0], TR.value[1]),
             'FD': "FD < {}".format(FD.value),
             'SNR': "SNR > {}".format(SNR.value)}

for item in to_filter:
    add_item=filter_dict.get(item)
    filter_list.append(add_item)
    
print(filter_list)

['TR > 2.1', 'FD < 0.5', 'SNR > 3.0']


In [61]:
# Arguments #

# laziness helper
# here = os.path.dirname(os.path.abspath(os.path.realpath(__file__)))
here = %pwd

# path to input of local data from MRIQC on your own dataset 
group_file = os.path.join(here,'test_data', 'group_bold.tsv')

# scan type to query the API for [bold, T1w, T2w]
modality = 'bold'

# any scan parameters that you want to filter the API search results by
"""Current possible filters:
   SNR, TSNR, DVAR, FD,
   FWHM, Tesla, gsr_x, gsr_y, TE, TR
   NOTE: Only working as *and* right now!
"""


# IQM variables to visualize
#IQM_to_plot = ['aor','aqi','dummy_trs','dvars_nstd','dvars_std','dvars_vstd',
                    #'efc','fber','fd_mean','fd_num','fd_perc','fwhm_avg','fwhm_x','fwhm_y',
                    #'fwhm_z','gcor','gsr_x','gsr_y','snr','summary_bg_k','summary_bg_mad',
                   # 'summary_bg_mean','summary_bg_median','summary_bg_n','summary_bg_p05',
                   # 'summary_bg_p95','summary_bg_stdv','summary_fg_k','summary_fg_mad',
                   # 'summary_fg_mean','summary_fg_median','summary_fg_n','summary_fg_p05',
                   # 'summary_fg_p95','summary_fg_stdv','tsnr']

'Current possible filters:\n   SNR, TSNR, DVAR, FD,\n   FWHM, Tesla, gsr_x, gsr_y, TE, TR\n   NOTE: Only working as *and* right now!\n'

In [62]:
# Load in your own data # 

# This should be a .csv or .tsv file outputted from MRIQC on your own data
# This will return a pandas dataframe of the MRIQC data from your experiment

userdf = load_groupfile(group_file)
# userdf.head()
userdf.shape

(18, 45)

In [63]:
# Load and filter data from the API # 

# Figure out which to get from modality arg #
T1apicsv = os.path.join(here, 'demo_api', 'T1w_demo.csv')
T2apicsv = os.path.join(here, 'demo_api', 'T2w_demo.csv')
boldapicsv = os.path.join(here, 'demo_api', 'bold_demo.csv')

if modality == 'T1w':
    api_file = T1apicsv
elif modality == 'T2w':
    api_file = T1apicsv
elif modality == 'bold':
    api_file = boldapicsv

# This will return a pandas dataframe with data from all scans of the given scan type
# with the given parameters 

apidf = pd.read_csv(api_file)
if not filter_list == []:
    filtered_apidf = filterIQM(apidf,filter_list)
else:
    filtered_apidf = apidf

# apidf.head()
print(apidf.shape)
# filtered_apidf.head()
print(filtered_apidf.shape)
#print(list(filtered_apidf))

(1000, 77)
(214, 77)


In [54]:
# Merge dataframes # 

# Takes the user data and API data and merges it into one dataframe 
# This will return a single pandas dataframe with the local data and API data merged, with a "group" measure to allow for a "groupby" 
# this needs to be updated with actual function name and information about how to use  

vis_ready_df = merge_dfs(userdf.copy(), filtered_apidf.copy())
#print(vis_ready_df.head())
#print(vis_ready_df.tail())
vis_ready_df.shape



(1018, 78)

In [38]:
# Visualization # 

# Create a violin plot of each MRIQC metric, comparing user level-data to API data 
# Also will print out a short description of the measure and what the appropriate range/values should be 
# Input the merged dataframe including the API and user data
# Optional arguments include a list of specific variables that the user wants displayed 
# If no optional arguments included, will plot all measures 

v = make_vio_plot(vis_ready_df,IQM_to_plot,"",outliers=True)

widgets.VBox([v[0],v[1]])


TypeError: make_vio_plot() got an unexpected keyword argument 'outliers'